In [1]:
import numpy as np
import os
import shutil
import tarfile
import tensorflow as tf
from transformers import BertTokenizer, TFBertForSequenceClassification
import pandas as pd
from bs4 import BeautifulSoup
import re
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.offline as pyo
import plotly.graph_objects as go
from wordcloud import WordCloud, STOPWORDS
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
from tqdm import tqdm

C:\Users\anili\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [2]:
from bert_modules import train_bert, Get_sentiment, get_sentiment_pretrained, make_predictions

In [3]:
from datasets import load_dataset

dataset = load_dataset("sentiment140")

dataset['train'][0]

C:\Users\anili\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\datasets\load.py:1486: FutureWarning:

The repository for sentiment140 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/sentiment140
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.



{'text': "@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D",
 'date': 'Mon Apr 06 22:19:45 PDT 2009',
 'user': '_TheSpecialOne_',
 'sentiment': 0,
 'query': 'NO_QUERY'}

In [4]:
df_pandas = pd.DataFrame(dataset["train"])
df_pandas.head()

,text,date,user,sentiment,query
0,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",Mon Apr 06 22:19:45 PDT 2009,_TheSpecialOne_,0,NO_QUERY
1,is upset that he can't update his Facebook by ...,Mon Apr 06 22:19:49 PDT 2009,scotthamilton,0,NO_QUERY
2,@Kenichan I dived many times for the ball. Man...,Mon Apr 06 22:19:53 PDT 2009,mattycus,0,NO_QUERY
3,my whole body feels itchy and like its on fire,Mon Apr 06 22:19:57 PDT 2009,ElleCTF,0,NO_QUERY
4,"@nationwideclass no, it's not behaving at all....",Mon Apr 06 22:19:57 PDT 2009,Karoli,0,NO_QUERY


In [5]:
from sklearn.utils import shuffle

df_small = shuffle(df_pandas, random_state=42).iloc[:18000]

df_small.shape
for i,data in df_small.iterrows():
    if data["sentiment"]==4: 
        df_small.at[i,"sentiment"]=1

df_small_train = df_small.iloc[:15000]
df_small_test = df_small.iloc[15000:]

df_small[df_small["sentiment"]==0].shape,df_small[df_small["sentiment"]==1].shape

((8962, 5), (9038, 5))

In [12]:
model, tokenizer = train_bert(df_small_train,df_small_test)

C:\Users\anili\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\bs4\__init__.py:435: MarkupResemblesLocatorWarning:

The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.

C:\Users\anili\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\bs4\__init__.py:435: MarkupResemblesLocatorWarning:

The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.

c:\Users\anili\OneDrive\Belgeler\GitHub\ml-ai-project\codes\bert_modules.py:34: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-co

Epoch 1/3
375/375 [==============================] - 2216s 6s/step - loss: 0.5082 - accuracy: 0.7437 - val_loss: 0.4268 - val_accuracy: 0.8060
Epoch 2/3
375/375 [==============================] - 2315s 6s/step - loss: 0.3491 - accuracy: 0.8550 - val_loss: 0.4344 - val_accuracy: 0.7990
Epoch 3/3
94/94 [==============================] - 127s 1s/step - loss: 0.5447 - accuracy: 0.8047
Test loss: 0.5446654558181763, Test accuracy: 0.8046666383743286


In [9]:
df_small_test['predicted_sentiment'] = make_predictions(df_small_test["text"], model, tokenizer)

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at models/Model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


94/94 [==============================] - 424s 4s/step


C:\Users\anili\AppData\Local\Temp\ipykernel_25936\288366473.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [11]:
from sklearn.metrics import classification_report

true_labels = df_small_test['sentiment']
predicted_labels = df_small_test['predicted_sentiment']

report = classification_report(true_labels, predicted_labels, target_names=["Negative", "Positive"])
print(report)

              precision    recall  f1-score   support

    Negative       0.81      0.80      0.81      1476
    Positive       0.81      0.82      0.81      1524

    accuracy                           0.81      3000
   macro avg       0.81      0.81      0.81      3000
weighted avg       0.81      0.81      0.81      3000



In [7]:
df_small_test["predicted_pretrained"] = get_sentiment_pretrained(df_small_test,  text="text", label="sentiment")

C:\Users\anili\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\bs4\__init__.py:435: MarkupResemblesLocatorWarning:

The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.

C:\Users\anili\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\torch\_utils.py:831: UserWarning:

TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()

100%|██████████| 3000/3000 [02:41<00:00, 18.56it/s]


accuracy:  0.767
precision:  0.7566894835096453
recall:  0.7979002624671916


C:\Users\anili\AppData\Local\Temp\ipykernel_37016\496689378.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [13]:
from sklearn.metrics import classification_report

true_labels = df_small_test['sentiment']
predicted_labels = df_small_test['predicted_pretrained']

report = classification_report(true_labels, predicted_labels, target_names=["Negative", "Positive"])
print(report)

              precision    recall  f1-score   support

    Negative       0.78      0.74      0.76      1476
    Positive       0.76      0.80      0.78      1524

    accuracy                           0.77      3000
   macro avg       0.77      0.77      0.77      3000
weighted avg       0.77      0.77      0.77      3000

